# Import libraries

In [1]:
import pandas as pd
from config import RAW_DIR, CLEAN_DIR
import numpy as np
import os

# Extract features and labels from the raw training meta-analysis file

In [2]:
# load up our training dataset
raw_training = pd.read_csv(os.path.join(RAW_DIR, "raw_training.csv"), index_col=[0, 3])
raw_training.iloc[:5, :20]

/Users/daniel/opt/anaconda3/envs/data/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10,11,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Author (year) BoiProject ID       RunID  \
Study_ID Sample Accession                                              
GMHI-23  SAMEA3879547      Vogtmann (2016)    PRJEB12449  ERR1293689   
         SAMEA3879551      Vogtmann (2016)    PRJEB12449  ERR1293705   
         SAMEA3879543      Vogtmann (2016)    PRJEB12449  ERR1293673   
         SAMEA3879565      Vogtmann (2016)    PRJEB12449  ERR1293761   
         SAMEA3879546      Vogtmann (2016)    PRJEB12449  ERR1293685   

                           Sequencing Platform Library layout (SE/PE)  \
Study_ID Sample Accession                                               
GMHI-23  SAMEA3879547      Illumina HiSeq 2000                 PAIRED   
         SAMEA3879551      Illumina HiSeq 2000                 PAIRED   
         SAMEA3879543      Illumina HiSeq 2000                 PAIRED   
         SAMEA3879565      Illumina HiSeq 2000                 PAIRED   
         SAMEA3879546      Illumina HiSeq 2000                 PAIRED   

                          Sample origin? (e.g., stool, mucosa) Disease status  \
Study_ID Sample Accession                                                       
GMHI-23  SAMEA3879547                                    stool        Control   
         SAMEA3879551                                    stool        Control   
         SAMEA3879543                                    stool        Control   
         SAMEA3879565                                    stool        Control   
         SAMEA3879546                                    stool        Control   

                                Sample_ID Subject Id (If available)  \
Study_ID Sample Accession                                             
GMHI-23  SAMEA3879547      MMRS84159866ST            MMRS84159866ST   
         SAMEA3879551      MMRS25211151ST            MMRS25211151ST   
         SAMEA3879543      MMRS32573774ST            MMRS32573774ST   
         SAMEA3879565      MMRS20257302ST            MMRS20257302ST   
         SAMEA3879546      MMRS86168210ST            MMRS86168210ST   

                          Age (Years)  Gender  \
Study_ID Sample Accession                       
GMHI-23  SAMEA3879547            67.0  Female   
         SAMEA3879551            60.0  Female   
         SAMEA3879543            82.0    Male   
         SAMEA3879565            62.0  Female   
         SAMEA3879546            79.0    Male   

                          Geographical Region or Population BMI (kg/m²)  \
Study_ID Sample Accession                                                 
GMHI-23  SAMEA3879547                                   USA    18.63574   
         SAMEA3879551                                   USA   20.466036   
         SAMEA3879543                                   USA   20.524438   
         SAMEA3879565                                   USA   20.665713   
         SAMEA3879546                                   USA   21.284602   

                          PHENOTYPE_Disease PHENOTYPE:Healthy_Nonhealthy  \
Study_ID Sample Accession                                                  
GMHI-23  SAMEA3879547               Healthy                      Healthy   
         SAMEA3879551               Healthy                      Healthy   
         SAMEA3879543               Healthy                      Healthy   
         SAMEA3879565               Healthy                      Healthy   
         SAMEA3879546               Healthy                      Healthy   

                             UNKNOWN  k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                                       
GMHI-23  SAMEA3879547      56.451568    0.861124                     0.861124   
         SAMEA3879551      39.905873    0.000000                     0.000000   
         SAMEA3879543      53.274915    0.151996                     0.151996   
         SAMEA3879565      59.044005    0.362985                     0.362985   
         SAMEA3879546      75.424715    0.001181         

In [4]:
raw_training["Author (year)"].unique()

array(['Vogtmann (2016)', 'Yu (2015)', 'He (2017)', 'Thomas (2019)',
       'Wirbel (2019)', 'Zhu (2018)', 'Franzosa (2018)', 'Feng (2015)',
       'Obregon-Tito (2015)', 'Schirmer (2016)',
       'HMP1 (Phase-I): Structure, function and diversity of the healthy human microbiome (HMP1-2012)',
       'HMP1 (Phase-II): Strains, functions and dynamics in the expanded Human Microbiome Project',
       'Zeevi (2015)', 'Nishijima (2016)', 'Jie (2017)', 'Li (2017)',
       'Backhed (2015)', 'Le Chatelier (2013)', 'Liu (2016)',
       'Qin (2014)', 'Qin (2012)', 'Karlsson (2013)', 'Nielsen (2014)',
       'Dhakan (2019)', 'Wen (2017)', 'Wilson (2021)', 'Xie (2016)',
       'Hansen (2018)', 'Weng (2019)', 'Lloyd-Price (2019)', 'Ye (2018)',
       'Tett (2019)', 'Costea (2017)', 'Roager (2019)', 'Zhu (2021)',
       'Hannigan (2018)', 'Wang (2021)', 'Yachida (2019)',
       'De Filippis (2019)', 'Winglee (2017)', 'Rampelli (2020)',
       'Mehta (2018)', 'Asnicar (2021)', 'Keohane (2020)',
     

In [27]:
print("Start with", raw_training.shape[0], "samples and", np.unique(raw_training.index.get_level_values(0)).shape[0], "studies")

Start with 12532 samples and 70 studies


In [28]:
# Select the taxonomic features
taxonomic_features = raw_training.iloc[:, 15:]

# Select the health status labels 
is_healthy = raw_training[["PHENOTYPE:Healthy_Nonhealthy"]] == "Healthy"

# Select the full phenotype labels
phenotype = raw_training[["PHENOTYPE_Disease"]]

# Apply sample exclusion criteria

In [29]:
non_illumina = [
    "454 GS FLX Titanium",
    "Ion Torrent PGM",
    "Ion Torrent Proton",
    "BGISEQ-500",
]

In [30]:
# Find the indices of samples we would like to remove

remove = (
    
    # we don't need a biomarker for obesity/underweight
    (phenotype == "Underweight").values.flatten() | 
    (phenotype == "Overweight").values.flatten() |
    (phenotype == "Obesity").values.flatten() |
    (phenotype == "Obese").values.flatten() |
    
    # to reduce batch effects, remove samples sequenced with non-illumina machines
    raw_training["Sequencing Platform"].isin(non_illumina).values.flatten() |
    
    # all of these studies have samples from subjects w/o a diagnosed disease but have an abonormal condition
    (phenotype.index.get_level_values(0) == "P4") | # P4 treats the poop for extracting viral DNA
    (phenotype.index.get_level_values(0) == "P86") | # Healthy at baseline but half develop T2D 
    (phenotype.index.get_level_values(0) == "GMHI-19") | # Outlier study
    ((phenotype.index.get_level_values(0) == "P48") & (phenotype == "Healthy").values.flatten()) | # Alcohol or smoking
    (phenotype.index.get_level_values(0) == "P59") | # Are all technically healthy, but half are in heavily urbanized areas
    # and "Microbes with higher relative abundance in Chinese urban samples have been associated with disease in other studies"
    (phenotype.index.get_level_values(0) == "P63") # Deals with semisupercentenarians, i.e., 105 to 109 years old
)

In [31]:
# remove bad samples
taxonomic_features = taxonomic_features.iloc[~remove, :]
is_healthy = is_healthy.iloc[~remove, :]
phenotype = phenotype.iloc[~remove, :]

## Remove samples w/ low read counts

In [32]:
# Get samples with low read counts
training_read_counts = pd.read_csv(os.path.join(RAW_DIR, "training_read_counts.csv"), index_col=[2]).iloc[:9045, :5].loc[taxonomic_features.index.get_level_values(1)]

# make sure indices align
assert((training_read_counts.index == phenotype.index.get_level_values(1)).mean() == 1)

In [33]:
# remove low quality samples
low_quality_index = (training_read_counts["Read_count"] < 1000000).values
taxonomic_features = taxonomic_features.iloc[~low_quality_index, :]
is_healthy = is_healthy.iloc[~low_quality_index, :]
phenotype = phenotype.iloc[~low_quality_index, :]

In [34]:
print(f"Our dataset has {taxonomic_features.shape[0]} samples with {taxonomic_features.shape[1]} unique taxnomic features")
print(f"Our dataset has {len(np.unique(taxonomic_features.index.get_level_values(0)))} independent studies")

Our dataset has 8904 samples with 3201 unique taxnomic features
Our dataset has 61 independent studies


# Apply study exclusion criteria

In [35]:
# Find studies with >= 20 samples
high_sample_studies = phenotype.groupby(level=0).count() >= 20
high_sample_studies = high_sample_studies[high_sample_studies["PHENOTYPE_Disease"]].index

# Keep only theses studies
taxonomic_features = taxonomic_features.loc[high_sample_studies]
is_healthy = is_healthy.loc[high_sample_studies]
phenotype = phenotype.loc[high_sample_studies]

In [36]:
# normalize relative abundances
taxonomic_features = taxonomic_features.divide((100 - taxonomic_features["UNKNOWN"]), axis="rows")

In [37]:
# we don't need unknown column anymore
taxonomic_features = taxonomic_features.drop(labels=["UNKNOWN"], axis=1)

# Check how many samples we have finally
print(f"Our dataset has {taxonomic_features.shape[0]} samples with {taxonomic_features.shape[1]} unique taxnomic features")
print(f"Our dataset has {len(np.unique(taxonomic_features.index.get_level_values(0)))} independent studies")


Our dataset has 8869 samples with 3200 unique taxnomic features
Our dataset has 57 independent studies


In [38]:
# there are 7 taxonomic ranks, each one should sum to 1, so sum of each sample should be 7 (w/ floating point error)
taxonomic_features.sum(axis=1)

Study_ID  Sample Accession
GMHI-10   SAMN03283239        6.998433
          SAMN03283266        6.999368
          SAMN03283281        6.774279
          SAMN03283245        6.994639
          SAMN03283294        6.997221
                                ...   
P98       SAMEA6104852        6.999441
          SAMEA6104851        6.999896
          SAMEA6104850        6.999830
          SAMEA6104885        6.999716
          SAMEA6104858        6.999882
Length: 8869, dtype: float64

# Standardize names for diseased phenotypes

In [39]:
np.unique(phenotype)

array(['ACVD', 'Adenoma (MP)', 'Behcet’s disease', 'Breast Cancer', 'CD',
       'CRC', 'Cancer', "Crohn''s disease", 'Crohns disease',
       'End-stage renal disease (ESRD)', 'Graves’ disease', 'Healthy',
       'Hypertension (HTN)', 'IGT', 'Large adenoma', 'Liver Cirrhosis',
       'NAFLD', 'Pancreatic cancer', 'Rheumatoid Arthritis',
       'Rheumatoid arthritis', 'Small adenoma', 'T2D',
       'Ulcerative colitis', 'adenoma', 'advanced adenoma',
       'ankylosing spondylitis', 'carcinoma', 'schizophrenia',
       'ulcerative colitis'], dtype=object)

In [40]:
dic = {
    "Adenoma (MP)" : "Colorectal adenoma",
    "Large adenoma" : "Colorectal adenoma",
    "Small adenoma" : "Colorectal adenoma",
    "adenoma" : "Colorectal adenoma",
    "advanced adenoma" : "Colorectal adenoma",
    "carcinoma" : "Colorectal adenoma",
    
    "Cancer" : "Colorectal cancer",
    "CRC" : "Colorectal cancer",
    
    "Crohn''s disease" : "Crohn's Disease",
    "CD" : "Crohn's Disease",
    "Crohns disease" : "Crohn's Disease",
    
    "Rheumatoid Arthritis" : "Rheumatoid arthritis",
    
    "IGT" : "Impaired glucose tolerance",
    
    "T2D" : "Type 2 diabetes",
    
    "schizophrenia" : "Schizophrenia",
    
    "ulcerative colitis" : "Ulcerative colitis",
    
    "ACVD" : "Atherosclerotic cardiovascular disease",
    
    "End-stage renal disease (ESRD)" : "End-stage renal disease",
    
    "Hypertension (HTN)" : "Hypertension",
    
    "ankylosing spondylitis" : "Ankylosing spondylitis",
    
}

phenotype = phenotype.replace({"PHENOTYPE_Disease" : dic}) 

In [41]:
# Look at how many studies and samples for each phenotype

phenotype_info = phenotype.reset_index().groupby("PHENOTYPE_Disease").agg(lambda x: x.nunique())
phenotype_info.columns = ["# of studies", "# of samples"]
phenotype_info

,# of studies,# of samples
PHENOTYPE_Disease,,
Ankylosing spondylitis,1,96
Atherosclerotic cardiovascular disease,1,214
Behcet’s disease,1,24
Breast Cancer,1,62
Colorectal adenoma,4,229
Colorectal cancer,6,558
Crohn's Disease,7,320
End-stage renal disease,1,223
Graves’ disease,1,100


# Yay we are done, export stuff

In [42]:
# export training set stuff
taxonomic_features.to_csv(os.path.join(CLEAN_DIR, "taxonomic_features.csv"))
is_healthy.to_csv(os.path.join(CLEAN_DIR, "is_healthy.csv"))
phenotype.to_csv(os.path.join(CLEAN_DIR, "phenotype.csv"))

# Clean validation data

## Clean first longitudinal data file

In [90]:
raw_val1 = pd.read_csv(os.path.join(RAW_DIR, "raw_validation.csv"), index_col=[0, 3])

In [87]:
# Select the taxonomic features
taxonomic_features_val1 = raw_val1.iloc[:, 14:]

# get meta
meta_val1 = raw_val1.iloc[:, :14]

# normalize and drop
taxonomic_features_val1 = taxonomic_features_val1.divide((100 - taxonomic_features_val1["UNKNOWN"]), axis="rows")
taxonomic_features_val1 = taxonomic_features_val1.drop(labels=["UNKNOWN"], axis=1)

# add dummy columns to align with training set
taxonomic_features_val1[list(set(taxonomic_features.columns) - set(taxonomic_features_val1.columns))] = 0
taxonomic_features_val1 = taxonomic_features_val1.copy()[taxonomic_features.columns]

# there are 7 taxonomic ranks, each one should sum to 1, so sum of each sample should be 7 (w/ floating point error)
taxonomic_features_val1.sum(axis=1)

/Users/daniel/opt/anaconda3/envs/data/lib/python3.7/site-packages/pandas/core/frame.py:3645: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = value


Study_ID  Sample Accession
P65       SAMEA104062441      6.999077
          SAMEA104062442      6.999857
          SAMEA104062443      7.000002
          SAMEA104062444      6.991888
          SAMEA104062445      6.996988
                                ...   
P116      SAMN12478559        6.999367
          SAMN12478560        6.999259
          SAMN12478561        6.995951
          SAMN12478591        6.999739
          SAMN12478594        6.996203
Length: 691, dtype: float64

## Clean second longitudinal data file

In [75]:
raw_val2 = pd.read_csv(os.path.join(RAW_DIR, "raw_validation2.csv"), index_col=[0, 4])

/Users/daniel/opt/anaconda3/envs/data/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [76]:
# Select the taxonomic features
taxonomic_features_val2 = raw_val2.iloc[:, 13:]

# get meta
meta_val2 = raw_val2.iloc[:, :13]


print("P90 original # of samples:", meta_val2.loc["P90"].shape[0])

P90 original # of samples: 428


### Remove samples w/ low read counts

In [77]:
# get low quality reads indices
validation2_read_counts = pd.read_csv(os.path.join(RAW_DIR, "validation2_read_counts.csv"), index_col=[2]).iloc[:672, :5].loc[taxonomic_features_val2.index.get_level_values(1)]

assert((validation2_read_counts.index == taxonomic_features_val2.index.get_level_values(1)).mean() == 1)

In [78]:
# remove low quality samples
low_quality_index_val2 = (validation2_read_counts["Read_count"] < 1000000).values
# taxonomic_features_val2 = taxonomic_features_val2.iloc[~low_quality_index_val2, :]
# meta_val2 = meta_val2.iloc[~low_quality_index_val2, :]

In [79]:
print("P90 final # of samples:", meta_val2.loc["P90"].shape[0])

P90 final # of samples: 428


In [80]:
# normalize and drop
taxonomic_features_val2 = taxonomic_features_val2.divide((100 - taxonomic_features_val2["UNKNOWN"]), axis="rows")
taxonomic_features_val2 = taxonomic_features_val2.drop(labels=["UNKNOWN"], axis=1)

# add dummy columns to align with training set
taxonomic_features_val2[list(set(taxonomic_features.columns) - set(taxonomic_features_val2.columns))] = 0
taxonomic_features_val2 = taxonomic_features_val2.copy()[taxonomic_features.columns]

# there are 7 taxonomic ranks, each one should sum to 1, so sum of each sample should be 7 (w/ floating point error)
taxonomic_features_val2.sum(axis=1)

/Users/daniel/opt/anaconda3/envs/data/lib/python3.7/site-packages/pandas/core/frame.py:3645: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = value


Study_ID  BioSample   
P107      SAMEA6512889    6.999663
          SAMEA6513017    6.996898
          SAMEA6513054    6.998531
          SAMEA6513021    6.994938
          SAMEA6512856    6.992086
                            ...   
P90       SAMN16701308    6.999814
          SAMN16701460    6.997841
          SAMN16701459    6.846649
          SAMN16701458    6.977201
          SAMN16701457    6.966260
Length: 672, dtype: float64

In [81]:
# Merge taxonomic features
taxonomic_features_val = pd.concat([taxonomic_features_val1, taxonomic_features_val2])

In [82]:
# Select columns of interest for metadata
meta_val1_cropped = meta_val1[["Subject_ID", "Health status", "Time point"]]
meta_val2_cropped = meta_val2[["patient_ID", "Host_diet", "timepoint"]]

In [83]:
# Rename column/index names
meta_val2_cropped.index = meta_val2_cropped.index.rename(["Study_ID", "Sample Accession"])
meta_val1_cropped.columns = ["Subject_ID", "health_status/host_diet", "timepoint"]
meta_val2_cropped.columns = ["Subject_ID", "health_status/host_diet", "timepoint"]

In [84]:
# Merge metadata 
metadata_val = pd.concat([meta_val1_cropped, meta_val2_cropped])
display(np.unique(metadata_val.index.get_level_values(0)))
metadata_val

array(['P106', 'P107', 'P116', 'P37', 'P43', 'P65', 'P89', 'P90'],
      dtype=object)

Subject_ID health_status/host_diet timepoint
Study_ID Sample Accession                                             
P65      SAMEA104062441       ERAS10                 Healthy        D0
         SAMEA104062442       ERAS10                 Healthy      D180
         SAMEA104062443       ERAS10                 Healthy       D42
         SAMEA104062444       ERAS10                 Healthy        D8
         SAMEA104062445       ERAS11                 Healthy        D0
...                              ...                     ...       ...
P90      SAMN16701308           9003                     EEN         3
         SAMN16701460           9017                   Vegan         1
         SAMN16701459           9016                     EEN        15
         SAMN16701458           9016                     EEN        14
         SAMN16701457           9016                     EEN        13

[1363 rows x 3 columns]

# Yay we are done, export stuff

In [85]:
taxonomic_features_val.to_csv(os.path.join(CLEAN_DIR, "taxonomic_features_val.csv"))
metadata_val.to_csv(os.path.join(CLEAN_DIR, "metadata_val.csv"))